# Project report - Continous Control

In this project the goal is to train a robotic arm to reach a  goal in space. Most work on reinforced learning agents are on discrete action spaces to e.g. learn how to play Atari games from pixels (like DQN - Deep Q-learning). The problem with robotic control is that the action space is continous - which is a more challenging task than discrete actions due to a possible infinite action space. 


## The DDPG algorithm

A algorithm called DDPG is developed to learn problems that has continous action spaces (controlling the motors and forces of the robot) and can be seen as DQN with continious actions. The robot model returns a state vector with all the info about the state of the robotic arm (not pixels) of 33 real values. The action vector (4 real values) is applied the model which replies with reward value and a new state vector. This is what we need to create a reinforced training agent that can learn the behaviour of the robotic arm without a model (model-free).

DDPG is an actor-critic method. The actor estimates the best actions to take and the critic estimates the Q(s, a) value for the state/action pairs. The Q(s, a) is learn using the Bellman equation as in Q-learning. 

The actor is updated




## Agent parameters

BUFFER_SIZE = int(1e6)  # replay buffer size
BATCH_SIZE = 256        # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR_ACTOR = 1e-3         # learning rate of the actor 
LR_CRITIC = 1e-3        # learning rate of the critic
WEIGHT_DECAY = 0        # L2 weight decay
EPSILON = 1.0           # epsilon for the noise process added to the actions
EPSILON_DECAY = 1e-6    # decay for epsilon above
NOISE_SIGMA = 0.05      # sigma for Ornstein-Uhlenbeck noise
UPDATE_EVERY = 20       # timesteps between updates
NUM_UPDATES = 10        # num of update passes when updating


## Nework parameters
The neural networks used in this solution two hidden layers of 256 nodes each. I tried with other sizes but this seems sufficent for training.

I added batch normalization between the layers with good effect on the speed of training (reaching goal in 133 vs. 199 epsisodes without batch normalization).


## Future improvements

There are ways to improve this even further by parallelizing the training using:
 * A2C
 * A3C
 
 * PPO
 